In [44]:
from pyspark.sql import SparkSession
import  pyspark.sql.functions as F
from pyspark.sql.functions import col, split, udf
from pyspark.sql.types import StructType as R, StructField as Fld, DoubleType as Dbl, StringType as Str, IntegerType as Int, DateType as Date
from pyspark.sql.types import StructType as R, StructField as Fld, DoubleType as Dbl, StringType as Str, IntegerType as Int, DateType as Date
from pyspark.sql.types import IntegerType, DateType
from pyspark.sql.functions import desc

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
df_spark=spark.read.parquet("s3://immigration-us-1/sas_data")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
df_spark.take(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(cicid=1817663.0, i94yr=2016.0, i94mon=4.0, i94cit=148.0, i94res=131.0, i94port='SFR', arrdate=20554.0, i94mode=1.0, i94addr='CA', depdate=20560.0, i94bir=23.0, i94visa=2.0, count=1.0, dtadfile='20160410', visapost=None, occup=None, entdepa='G', entdepd='O', entdepu=None, matflag='M', biryear=1993.0, dtaddto='07082016', gender='F', insnum=None, airline='LX', admnum=55960415633.0, fltno='00038', visatype='WT'), Row(cicid=1817664.0, i94yr=2016.0, i94mon=4.0, i94cit=148.0, i94res=131.0, i94port='SFR', arrdate=20554.0, i94mode=1.0, i94addr='CA', depdate=20561.0, i94bir=29.0, i94visa=1.0, count=1.0, dtadfile='20160410', visapost=None, occup=None, entdepa='G', entdepd='O', entdepu=None, matflag='M', biryear=1987.0, dtaddto='07082016', gender='M', insnum=None, airline='LX', admnum=55960341333.0, fltno='00038', visatype='WB'), Row(cicid=1817665.0, i94yr=2016.0, i94mon=4.0, i94cit=148.0, i94res=131.0, i94port='SFR', arrdate=20554.0, i94mode=1.0, i94addr='CA', depdate=20563.0, i94bir=47.0, i

In [47]:
df_spark.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3096313

In [48]:
immigrants=df_spark.where(F.col("i94mode")==1) 
immigrants.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2994505

In [49]:
immigrants=immigrants.select("cicid", "i94yr", "i94mon", "i94cit", "i94res", "i94port", "arrdate", "i94visa",  "biryear", "gender", "visatype", "airline")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
immigrants = immigrants.withColumn("biryear", immigrants["biryear"].cast(IntegerType()))
immigrants.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+------+------+------+------+-------+-------+-------+-------+------+--------+-------+
|    cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94visa|biryear|gender|visatype|airline|
+---------+------+------+------+------+-------+-------+-------+-------+------+--------+-------+
|1817663.0|2016.0|   4.0| 148.0| 131.0|    SFR|20554.0|    2.0|   1993|     F|      WT|     LX|
|1817664.0|2016.0|   4.0| 148.0| 131.0|    SFR|20554.0|    1.0|   1987|     M|      WB|     LX|
|1817665.0|2016.0|   4.0| 148.0| 131.0|    SFR|20554.0|    1.0|   1969|     M|      WB|     LX|
|1817666.0|2016.0|   4.0| 148.0| 131.0|    SFR|20554.0|    2.0|   1979|     F|      WT|     LX|
|1817667.0|2016.0|   4.0| 148.0| 131.0|    SFR|20554.0|    2.0|   1975|     M|      WT|     LX|
+---------+------+------+------+------+-------+-------+-------+-------+------+--------+-------+
only showing top 5 rows

In [51]:
Schema_country = R([
    Fld("id",Str()),
    Fld("country",Str())
    ])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
country_dict=spark.read.csv(
    "s3://immigration-us-1/raw_data/country_dict.csv", header=True, mode="DROPMALFORMED", sep="=",schema=Schema_country)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
country_dict.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---------------+
|     id|        country|
+-------+---------------+
|   236 |  'AFGHANISTAN'|
|   101 |      'ALBANIA'|
|   316 |      'ALGERIA'|
|   102 |      'ANDORRA'|
|   324 |       'ANGOLA'|
+-------+---------------+
only showing top 5 rows

In [54]:
spaceDeleteUDF = udf(lambda s: s.replace(" ", ""), Str())
ampesandDeleteUDF = udf(lambda s: s.replace("'", ""), Str())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
country_dict.withColumn("id", spaceDeleteUDF("id"))
country_dict.withColumn("country", spaceDeleteUDF("country"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[id: string, country: string]

In [56]:
country_dict.take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(id='   236 ', country="  'AFGHANISTAN'"), Row(id='   101 ', country="  'ALBANIA'"), Row(id='   316 ', country="  'ALGERIA'"), Row(id='   102 ', country="  'ANDORRA'"), Row(id='   324 ', country="  'ANGOLA'")]

In [57]:
immigrants = immigrants.join(country_dict, immigrants.i94cit == country_dict.id,how='right') 
immigrants=immigrants.withColumnRenamed("country", "cit_country")
immigrants=immigrants.drop('id','i94cit')
immigrants.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+------+------+------+-------+-------+-------+-------+------+--------+-------+------------+
|    cicid| i94yr|i94mon|i94res|i94port|arrdate|i94visa|biryear|gender|visatype|airline| cit_country|
+---------+------+------+------+-------+-------+-------+-------+------+--------+-------+------------+
|1859625.0|2016.0|   4.0| 299.0|    LOS|20554.0|    1.0|   1972|  null|      B1|     KE|  'MONGOLIA'|
|1859626.0|2016.0|   4.0| 299.0|    LOS|20554.0|    1.0|   1976|  null|      B1|     KE|  'MONGOLIA'|
|1859627.0|2016.0|   4.0| 299.0|    CHI|20554.0|    1.0|   1968|  null|      B1|     AB|  'MONGOLIA'|
|1859628.0|2016.0|   4.0| 299.0|    SFR|20554.0|    3.0|   1989|     F|      F1|     UA|  'MONGOLIA'|
|1859629.0|2016.0|   4.0| 299.0|    SFR|20554.0|    3.0|   2011|     M|      F2|     UA|  'MONGOLIA'|
|1859630.0|2016.0|   4.0| 299.0|    LVG|20554.0|    1.0|   1966|     M|      B1|     KE|  'MONGOLIA'|
|1859633.0|2016.0|   4.0| 299.0|    DAL|20554.0|    2.0|   1965|     M|      B2|  

In [58]:
immigrants = immigrants.join(country_dict, immigrants.i94res == country_dict.id,how='right') 
immigrants=immigrants.withColumnRenamed("country", "res_country")
immigrants=immigrants.drop('id','i94res')
immigrants.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+------+------+-------+-------+-------+-------+------+--------+-------+------------+------------+
|    cicid| i94yr|i94mon|i94port|arrdate|i94visa|biryear|gender|visatype|airline| cit_country| res_country|
+---------+------+------+-------+-------+-------+-------+------+--------+-------+------------+------------+
|5761447.0|2016.0|   4.0|    LOS|20574.0|    2.0|   1972|     F|      B2|     KE|  'MONGOLIA'|  'MONGOLIA'|
|5761448.0|2016.0|   4.0|    CHI|20574.0|    2.0|   1974|     F|      B2|     SK|  'MONGOLIA'|  'MONGOLIA'|
|5761451.0|2016.0|   4.0|    WAS|20574.0|    3.0|   1982|     M|      F2|     UA|  'MONGOLIA'|  'MONGOLIA'|
|5761452.0|2016.0|   4.0|    WAS|20574.0|    2.0|   1990|     F|      B2|     UA|  'MONGOLIA'|  'MONGOLIA'|
|5761453.0|2016.0|   4.0|    WAS|20574.0|    1.0|   1966|     M|      B1|     KE|  'MONGOLIA'|  'MONGOLIA'|
|5761454.0|2016.0|   4.0|    WAS|20574.0|    2.0|   1950|     F|      B2|     KE|  'MONGOLIA'|  'MONGOLIA'|
|5761455.0|2016.0|   4.0|   

In [59]:
Schema_Visa = R([
    Fld("id",Str()),
    Fld("Visa_Type",Str())
    ])
visa_dict=spark.read.csv(
    "s3://immigration-us-1/raw_data/visa_dict.csv", header=False, mode="DROPMALFORMED",sep="=", schema=Schema_Visa)   

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
visa_dict.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+---------+
| id|Visa_Type|
+---+---------+
| 1 | Business|
| 2 | Pleasure|
| 3 |  Student|
+---+---------+

In [61]:
immigrants = immigrants.join(visa_dict, immigrants.i94visa == visa_dict.id,how='right') 
immigrants=immigrants.drop('id','i94visa')
immigrants.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+------+------+-------+-------+-------+------+--------+-------+-----------+-------------+---------+
|    cicid| i94yr|i94mon|i94port|arrdate|biryear|gender|visatype|airline|cit_country|  res_country|Visa_Type|
+---------+------+------+-------+-------+-------+------+--------+-------+-----------+-------------+---------+
|5748517.0|2016.0|   4.0|    LOS|20574.0|   1976|     F|      B1|     QF|    'CHINA'|  'AUSTRALIA'| Business|
|5748518.0|2016.0|   4.0|    LOS|20574.0|   1984|     F|      B1|     VA|    'CHINA'|  'AUSTRALIA'| Business|
|5748519.0|2016.0|   4.0|    LOS|20574.0|   1987|     M|      B1|     DL|    'CHINA'|  'AUSTRALIA'| Business|
|5748520.0|2016.0|   4.0|    LOS|20574.0|   1987|     F|      B1|     DL|    'CHINA'|  'AUSTRALIA'| Business|
|5748521.0|2016.0|   4.0|    LOS|20574.0|   1988|     M|      B1|     DL|    'CHINA'|  'AUSTRALIA'| Business|
|5748889.0|2016.0|   4.0|    ORL|20574.0|   1988|     M|      B1|     B6|    'CHINA'|   'COLOMBIA'| Business|
|5748904.0

In [17]:
#immigrants.createOrReplaceTempView("immigrants")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
#immigrants.createOrReplaceTempView("immigrants")
#immigrants = spark.sql("SELECT * FROM immigrants")
#immigrants=immigrants.drop('arrdate')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
#immigrants.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [65]:
#immigrants = immigrants.withColumn("arrdate", immigrants["arrdate"].cast(IntegerType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
immigrants.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+------+------+-------+-------+-------+------+--------+-------+-----------+-------------+---------+
|    cicid| i94yr|i94mon|i94port|arrdate|biryear|gender|visatype|airline|cit_country|  res_country|Visa_Type|
+---------+------+------+-------+-------+-------+------+--------+-------+-----------+-------------+---------+
|5748517.0|2016.0|   4.0|    LOS|20574.0|   1976|     F|      B1|     QF|    'CHINA'|  'AUSTRALIA'| Business|
|5748518.0|2016.0|   4.0|    LOS|20574.0|   1984|     F|      B1|     VA|    'CHINA'|  'AUSTRALIA'| Business|
|5748519.0|2016.0|   4.0|    LOS|20574.0|   1987|     M|      B1|     DL|    'CHINA'|  'AUSTRALIA'| Business|
|5748520.0|2016.0|   4.0|    LOS|20574.0|   1987|     F|      B1|     DL|    'CHINA'|  'AUSTRALIA'| Business|
|5748521.0|2016.0|   4.0|    LOS|20574.0|   1988|     M|      B1|     DL|    'CHINA'|  'AUSTRALIA'| Business|
|5748889.0|2016.0|   4.0|    ORL|20574.0|   1988|     M|      B1|     B6|    'CHINA'|   'COLOMBIA'| Business|
|5748904.0

In [63]:
import datetime as dt
def to_datetime(x):
    try:
        start = dt.datetime(1960, 1, 1).date()
        return start + dt.timedelta(days=int(x))
    except:
        return None
udf_to_datetime_sas = udf(lambda x: to_datetime(x), DateType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [64]:
immigrants=immigrants.withColumn("arrdate", udf_to_datetime_sas("arrdate"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [65]:
immigrants.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+------+------+-------+----------+-------+------+--------+-------+-----------+-------------+---------+
|    cicid| i94yr|i94mon|i94port|   arrdate|biryear|gender|visatype|airline|cit_country|  res_country|Visa_Type|
+---------+------+------+-------+----------+-------+------+--------+-------+-----------+-------------+---------+
|5748517.0|2016.0|   4.0|    LOS|2016-04-30|   1976|     F|      B1|     QF|    'CHINA'|  'AUSTRALIA'| Business|
|5748518.0|2016.0|   4.0|    LOS|2016-04-30|   1984|     F|      B1|     VA|    'CHINA'|  'AUSTRALIA'| Business|
|5748519.0|2016.0|   4.0|    LOS|2016-04-30|   1987|     M|      B1|     DL|    'CHINA'|  'AUSTRALIA'| Business|
|5748520.0|2016.0|   4.0|    LOS|2016-04-30|   1987|     F|      B1|     DL|    'CHINA'|  'AUSTRALIA'| Business|
|5748521.0|2016.0|   4.0|    LOS|2016-04-30|   1988|     M|      B1|     DL|    'CHINA'|  'AUSTRALIA'| Business|
|5748889.0|2016.0|   4.0|    ORL|2016-04-30|   1988|     M|      B1|     B6|    'CHINA'|   'COLO

In [66]:
immigrants = immigrants.withColumn("cicid", immigrants["cicid"].cast(IntegerType()))
immigrants = immigrants.withColumn("i94yr", immigrants["i94yr"].cast(IntegerType()))
immigrants = immigrants.withColumn("i94mon", immigrants["i94mon"].cast(IntegerType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
immigrants.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- cicid: integer (nullable = true)
 |-- i94yr: integer (nullable = true)
 |-- i94mon: integer (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: date (nullable = true)
 |-- biryear: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- visatype: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- cit_country: string (nullable = true)
 |-- res_country: string (nullable = true)
 |-- Visa_Type: string (nullable = true)

In [34]:
immigrants.write.parquet("s3://immigration-us-1/sas_data_ready_to_use")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
immigrants.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2444372

In [69]:
imm=immigrants.limit(1000)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
imm.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1000

In [33]:
imm.write.parquet("s3://immigration-us-1/_sas_data_ready_to_use")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Final script.

In [43]:
from pyspark.sql import SparkSession
import  pyspark.sql.functions as F
from pyspark.sql.functions import col, split, udf
from pyspark.sql.types import StructType as R, StructField as Fld, DoubleType as Dbl, StringType as Str, IntegerType as Int, DateType as Date
from pyspark.sql.types import StructType as R, StructField as Fld, DoubleType as Dbl, StringType as Str, IntegerType as Int, DateType as Date
from pyspark.sql.types import IntegerType, DateType
from pyspark.sql.functions import desc
import datetime as dt

def date_converter(x):
    try:
        temp = dt.datetime(1960, 1, 1).date()
        return temp + dt.timedelta(days=int(x))
    except:
        return None

    
def immigration_etl(source="s3://immigration-us-1/sas_data",
                    destination="s3://immigration-us-1/sas_data_ready_to_use",
                    country_dict="s3://immigration-us-1/raw_data/country_dict.csv",
                    visa_dict="s3://immigration-us-1/raw_data/visa_dict.csv"
                   ):
    # Local udfs
    
    spaceDeleteUDF = udf(lambda s: s.replace(" ", ""), Str())
    ampesandDeleteUDF = udf(lambda s: s.replace("'", ""), Str())
    udf_to_datetime_sas = udf(lambda x: date_converter(x), DateType())
   
    Schema_country = R([
        Fld("id",Str()),
        Fld("country",Str())
        ])
    Schema_Visa = R([
        Fld("id",Str()),
        Fld("Visa_Type",Str())
        ])

    df_spark=spark.read.parquet(source)
    # Only immigrants from air 
    immigrants=df_spark.where(F.col("i94mode")==1)
    immigrants=immigrants.select("cicid", "i94yr", "i94mon", "i94cit", "i94res", "i94port", "arrdate", "i94visa",  "biryear", "gender", "visatype", "airline")
    country_dict=spark.read.csv(country_dict, header=True, mode="DROPMALFORMED", sep="=",schema=Schema_country)
    
    immigrants = immigrants.join(country_dict, immigrants.i94cit == country_dict.id,how='right') 
    immigrants=immigrants.withColumnRenamed("country", "cit_country")
    immigrants=immigrants.drop('id','i94cit')
    
    immigrants = immigrants.join(country_dict, immigrants.i94res == country_dict.id,how='right') 
    immigrants=immigrants.withColumnRenamed("country", "res_country")
    immigrants=immigrants.drop('id','i94res')
    
    visa_dict=spark.read.csv(visa_dict, header=False, mode="DROPMALFORMED",sep="=", schema=Schema_Visa)
    immigrants = immigrants.join(visa_dict, immigrants.i94visa == visa_dict.id,how='right') 
    immigrants=immigrants.drop('id','i94visa')
    
    immigrants=immigrants.withColumn("arrdate", udf_to_datetime_sas("arrdate"))
    
    immigrants = immigrants.withColumn("cicid", immigrants["cicid"].cast(IntegerType()))
    immigrants = immigrants.withColumn("i94yr", immigrants["i94yr"].cast(IntegerType()))
    immigrants = immigrants.withColumn("biryear", immigrants["biryear"].cast(IntegerType()))
    immigrants = immigrants.withColumn("i94mon", immigrants["i94mon"].cast(IntegerType()))
    
    immigrants.show(10) 
    immigrants.write.parquet(destination)
   
    
immigration_etl()   

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-----+------+-------+----------+-------+------+--------+-------+------------------+------------------+---------+
| cicid|i94yr|i94mon|i94port|   arrdate|biryear|gender|visatype|airline|       cit_country|       res_country|Visa_Type|
+------+-----+------+-------+----------+-------+------+--------+-------+------------------+------------------+---------+
|459657| 2016|     4|    ATL|2016-04-03|   1972|     M|      WB|     BA|  'UNITED KINGDOM'|  'UNITED KINGDOM'| Business|
|459658| 2016|     4|    ATL|2016-04-03|   1977|     M|      WB|     BA|  'UNITED KINGDOM'|  'UNITED KINGDOM'| Business|
|459663| 2016|     4|    ATL|2016-04-03|   1945|     M|      WB|     DL|  'UNITED KINGDOM'|  'UNITED KINGDOM'| Business|
|459665| 2016|     4|    ATL|2016-04-03|   1962|     M|      WB|     BA|  'UNITED KINGDOM'|  'UNITED KINGDOM'| Business|
|459666| 2016|     4|    ATL|2016-04-03|   1965|     M|      WB|     BA|  'UNITED KINGDOM'|  'UNITED KINGDOM'| Business|
|459667| 2016|     4|    ATL|201